In [1]:
%reload_ext tensorboard
import datetime

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.losses as losses
import tensorflow.keras.metrics as metrics
from tensorflow.keras import regularizers
import tensorflow.keras.optimizers as optimizers

import tensorflow_datasets as tfds

from tensorflow.data import AUTOTUNE

C:\Users\andre\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\andre\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\andre\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3000)])
  except RuntimeError as e:
    print(e)

Found GPU at: /device:GPU:0


In [3]:
import json
import tensorflow as tf
import numpy as np
import cv2


seed = 123

Crop_img = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomCrop(128, 256, seed=seed),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])
Crop_seg= tf.keras.Sequential([
  layers.experimental.preprocessing.RandomCrop(128, 256, seed=seed),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])

seed = 456

Zoom_img = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomZoom((-0.75,0.25), fill_mode='constant', seed = seed),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])
Zoom_seg = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomZoom((-0.75,0.25), fill_mode='constant', seed = seed),
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])

seed= 789

Flip_img = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])
Flip_seg = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',seed = seed)
])

INPUT_SHAPE = (128,256,3)
OUTPUT_SHAPE = (128,256,2)

def class_map_road(seg):
    # map class 0=anything, 1=road
    return tf.where(seg == 7, [0, 1.0], [1.0, 0])


def cityscapes_prep(output_shape, input_shape=INPUT_SHAPE, class_map_func=None, float_range=True, standardize=True):
    def prep_map(sample):
        img = sample['image_left']
        seg = sample['segmentation_label']

        if float_range:
            img /= 255
        if standardize:
            img = tf.image.per_image_standardization(img)

        img = tf.image.resize(img, input_shape[0:2], method='bilinear')
        seg = tf.image.resize(seg, output_shape[0:2], method='nearest')
        
        img = img[None, ...]
        seg = seg[None, ...]
        img = Flip_img(img)
        seg = Flip_seg(seg)
        img = tf.squeeze(img)
        seg = tf.squeeze(seg,0)
        
        if callable(class_map_func):
            seg = class_map_func(seg)
        else:
            seg = tf.one_hot(tf.cast(seg, tf.int32), output_shape[-1], axis=2)
            seg = tf.cast(seg, tf.float32)
            seg = tf.squeeze(seg)
            #seg = tf.keras.utils.to_categorical(seg, num_classes=output_shape[-1])

        return img, seg

    return prep_map

def Zoom_prep(output_shape, input_shape=INPUT_SHAPE, class_map_func=None, float_range=True, standardize=True):
    def prep_map(sample):
        img = sample['image_left']
        seg = sample['segmentation_label']

        if float_range:
            img /= 255
        if standardize:
            img = tf.image.per_image_standardization(img)

        img = tf.image.resize(img, input_shape[0:2], method='bilinear')
        seg = tf.image.resize(seg, output_shape[0:2], method='nearest')
        
        img = img[None, ...]
        seg = seg[None, ...]
        img = Zoom_img(img)
        seg = Zoom_seg(seg)
        img = tf.squeeze(img)
        seg = tf.squeeze(seg,0)
        
        if callable(class_map_func):
            seg = class_map_func(seg)
        else:
            seg = tf.one_hot(tf.cast(seg, tf.int32), output_shape[-1], axis=2)
            seg = tf.cast(seg, tf.float32)
            seg = tf.squeeze(seg)
            #seg = tf.keras.utils.to_categorical(seg, num_classes=output_shape[-1])
        print(seg)
        return img, seg

    return prep_map

def Crop_prep(output_shape, input_shape=INPUT_SHAPE, class_map_func=None, float_range=True, standardize=True, Training = True):
    def prep_map(sample):
        img = sample['image_left']
        seg = sample['segmentation_label']

        if float_range:
            img /= 255
        if standardize:
            img = tf.image.per_image_standardization(img)

        img = tf.image.resize(img, (200,400), method='bilinear')
        seg = tf.image.resize(seg, (200,400), method='nearest')
                    
        
        img = img[None, ...]
        seg = seg[None, ...]
        img = Crop_img(img)
        seg = Crop_seg(seg)
        img = tf.squeeze(img)
        seg = tf.squeeze(seg,0)
        
        if callable(class_map_func):
            seg = class_map_func(seg)
        else:
            seg = tf.one_hot(tf.cast(seg, tf.int32), output_shape[-1], axis=2)
            seg = tf.cast(seg, tf.float32)
            seg = tf.squeeze(seg)
            #seg = tf.keras.utils.to_categorical(seg, num_classes=output_shape[-1])
        
        return img, seg

    return prep_map

rng = tf.random.Generator.from_seed(123, alg='philox')

def augment(img, seg, seed):
        #seed = tf.random.uniform((2,))
        print(seg)
        img = tf.image.stateless_random_brightness(img, 0.2, seed=seed)
        img = tf.image.stateless_random_contrast(img, 0.8, 1.2, seed=seed)
        img = tf.image.stateless_random_saturation(img, 0.0, 1.5, seed=seed)
        img = tf.image.stateless_random_hue(img, 0.2, seed=seed)
        img = tf.clip_by_value(img, 0.0, 1.0) # clip values outside 0..1

        return img, seg

def randomize(img, seg):
        seed = rng.make_seeds(2)[0]
        img, seg = augment(img, seg, seed)
        return img, seg

class ArgmaxMeanIOU(metrics.MeanIoU):
    def update_state(self, y_true, y_pred, sample_weight=None):
        return super().update_state(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1), sample_weight)

In [4]:
tf.random.set_seed(1234)

cityscapes = tfds.load('cityscapes/semantic_segmentation',data_dir="E:\EE\project\FPGA\cityscapes",download=False,shuffle_files=True)
train_ds = cityscapes['train'].map(cityscapes_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
valid_ds = cityscapes['validation'].map(cityscapes_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
test_ds = cityscapes['test'].map(cityscapes_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
crop_train = cityscapes['train'].map(Crop_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
zoom_train = cityscapes['train'].map(Zoom_prep(OUTPUT_SHAPE, INPUT_SHAPE, class_map_road))
fit_ds = train_ds.concatenate(crop_train).concatenate(zoom_train)

fit_ds = fit_ds.batch(16).prefetch(AUTOTUNE)
valid_ds = valid_ds.batch(4).prefetch(AUTOTUNE)
test_ds = test_ds.batch(4).prefetch(AUTOTUNE)
print(fit_ds)

Tensor("SelectV2:0", shape=(128, 256, 2), dtype=float32)
<PrefetchDataset shapes: ((None, 128, 256, 3), (None, 128, 256, 2)), types: (tf.float32, tf.float32)>


In [7]:
import tensorflow_model_optimization as tfmot
import tensorflow as tf
import tensorflow_addons as tfa


model = tf.keras.models.load_model('E:/EE/project/FPGA/benchmark_model/model_benchmark.tf',custom_objects={'ArgmaxMeanIOU': ArgmaxMeanIOU}, compile=False)

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

epochs = 35

NUM_CLASSES = 2

end_step = np.ceil(len(train_ds)).astype(np.int32)*epochs

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.2,
                                    final_sparsity=0.8,
                                    begin_step=0,
                                    end_step=end_step)}

pruned_model = prune_low_magnitude(model, **pruning_params)

sgd = optimizers.SGD(
        learning_rate=1e-5,
        momentum=0.9,
    )

opt = tfa.optimizers.MovingAverage(sgd, dynamic_decay=True)
cce = losses.CategoricalCrossentropy(from_logits=True)

pruned_model.compile(opt, loss=cce, metrics=['accuracy', ArgmaxMeanIOU(NUM_CLASSES)]) 
pruned_model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 256, 3) 0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv2d (Pru (None, 64, 128, 16)  882         input_1[0][0]                    
__________________________________________________________________________________________________
prune_low_magnitude_batch_norma (None, 64, 128, 16)  65          prune_low_magnitude_conv2d[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_activation  (None, 64, 128, 16)  1           prune_low_magnitude_batch_normali
______________________________________________________________________________________________

In [8]:
import tempfile

logdir = tempfile.mkdtemp()
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]   
history = pruned_model.fit(fit_ds,
                validation_data=valid_ds,
                epochs=epochs,
                callbacks=callbacks) 
pruned_model.save('E:/EE/project/FPGA/benchmark_model/pruned_model.tf')

Epoch 1/35
558/558 [==============================] - 947s 2s/step - loss: 0.0787 - accuracy: 0.9757 - argmax_mean_iou_1: 0.9489 - val_loss: 0.1184 - val_accuracy: 0.9638 - val_argmax_mean_iou_1: 0.9216
Epoch 2/35
558/558 [==============================] - 937s 2s/step - loss: 0.0780 - accuracy: 0.9760 - argmax_mean_iou_1: 0.9495 - val_loss: 0.1288 - val_accuracy: 0.9618 - val_argmax_mean_iou_1: 0.9174
Epoch 3/35
558/558 [==============================] - 983s 2s/step - loss: 0.0777 - accuracy: 0.9762 - argmax_mean_iou_1: 0.9498 - val_loss: 0.1262 - val_accuracy: 0.9631 - val_argmax_mean_iou_1: 0.9201
Epoch 4/35
558/558 [==============================] - 969s 2s/step - loss: 0.0784 - accuracy: 0.9757 - argmax_mean_iou_1: 0.9490 - val_loss: 0.1318 - val_accuracy: 0.9619 - val_argmax_mean_iou_1: 0.9176
Epoch 5/35
558/558 [==============================] - 996s 2s/step - loss: 0.0783 - accuracy: 0.9758 - argmax_mean_iou_1: 0.9490 - val_loss: 0.1213 - val_accuracy: 0.9635 - val_argmax_mean

C:\Users\andre\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: E:/EE/project/FPGA/benchmark_model/pruned_model.tf\assets


INFO:tensorflow:Assets written to: E:/EE/project/FPGA/benchmark_model/pruned_model.tf\assets


In [11]:
stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)
stripped_pruned_model.save("E:/EE/project/FPGA/benchmark_model/stripped_pruned_model.h5",save_format='h5')
stripped_pruned_model_copy = tf.keras.models.clone_model(stripped_pruned_model)
stripped_pruned_model_copy.set_weights(stripped_pruned_model.get_weights())

In [13]:
import tensorflow_model_optimization as tfmot

# Clustering
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization
from tensorflow_model_optimization.python.core.clustering.keras.experimental import (
    cluster,
)

cluster_weights = cluster.cluster_weights

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.DENSITY_BASED,
  'preserve_sparsity': True
}

sparsity_clustered_model = cluster_weights(stripped_pruned_model_copy, **clustering_params)

sparsity_clustered_model.compile(opt, loss=cce,
                  metrics=['accuracy', ArgmaxMeanIOU(NUM_CLASSES)])

print('Train sparsity preserving clustering model:')
sparsity_clustered_model.fit(fit_ds, validation_data=valid_ds, epochs=10)
sparsity_clustered_model.save("E:/EE/project/FPGA/benchmark_model/clustered_model.tf")

Train sparsity preserving clustering model:
Epoch 1/10
558/558 [==============================] - 882s 2s/step - loss: 0.3296 - accuracy: 0.8760 - argmax_mean_iou_2: 0.7613 - val_loss: 0.2390 - val_accuracy: 0.9304 - val_argmax_mean_iou_2: 0.8547
Epoch 2/10
558/558 [==============================] - 869s 2s/step - loss: 0.2361 - accuracy: 0.9406 - argmax_mean_iou_2: 0.8799 - val_loss: 0.2294 - val_accuracy: 0.9357 - val_argmax_mean_iou_2: 0.8646
Epoch 3/10
558/558 [==============================] - 865s 2s/step - loss: 0.2296 - accuracy: 0.9456 - argmax_mean_iou_2: 0.8898 - val_loss: 0.2237 - val_accuracy: 0.9394 - val_argmax_mean_iou_2: 0.8716
Epoch 4/10
558/558 [==============================] - 887s 2s/step - loss: 0.2223 - accuracy: 0.9486 - argmax_mean_iou_2: 0.8952 - val_loss: 0.2195 - val_accuracy: 0.9420 - val_argmax_mean_iou_2: 0.8771
Epoch 5/10
558/558 [==============================] - 877s 2s/step - loss: 0.2216 - accuracy: 0.9494 - argmax_mean_iou_2: 0.8969 - val_loss: 0.2

C:\Users\andre\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: E:/EE/project/FPGA/benchmark_model/clustered_model.tf\assets


INFO:tensorflow:Assets written to: E:/EE/project/FPGA/benchmark_model/clustered_model.tf\assets


In [ ]:
stripped_sparsity_clustered_model = tfmot.clustering.keras.strip_clustering(sparsity_clustered_model)
stripped_sparsity_clustered_model.save("E:/EE/project/FPGA/benchmark_model/stripped_clustered_model.tf")